In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


dataset_path = '..\data\Datasets_SVFS\colon.csv'
# dataset_path = '/content/drive/My Drive/data_code_DSCI6601/data/Datasets_SVFS/SMK_CAN_187.csv'

if not os.path.exists(dataset_path):
    print(f"File not found: {dataset_path}")
else:
    # Read the dataset from a CSV file
    data = pd.read_csv(dataset_path, header=None)

    # Print dataset specifications
    print("Dataset Specifications:")
    print(f"Number of samples: {data.shape[0]}")
    print(f"Number of columns: {data.shape[1]}")
    # print(f"Number of labels: {len(data.select_dtypes(include=['object', 'category']).columns)}")
    # print(f"Column names: {data.columns.tolist()}")
    print("\nData Preview:")
    print(data.head())

    # Check for normalization requirement (if any feature has variance > 1, normalization is needed)
    # feature_columns = data.select_dtypes(include=[np.number]).columns.tolist()
    # features = data[feature_columns]
    X = data.iloc[:, :-1]  # All columns except the last column
    y = data.iloc[:, -1]  # The last column is assumed to be the target label

    # Print the number of samples per label type
    label_counts = y.value_counts()
    print("\nNumber of samples per label type:")
    print(label_counts)
    print("\nChecking if normalization is required...")

Dataset Specifications:
Number of samples: 62
Number of columns: 2001

Data Preview:
   0     1     2     3     4     5     6     7     8     9     ...  1991  \
0   2.0   0.0   0.0   0.0  -2.0   0.0  -2.0   0.0   2.0   0.0  ...   0.0   
1   2.0   2.0   0.0   0.0  -2.0   0.0   0.0   0.0   2.0   0.0  ...   0.0   
2  -2.0   2.0   2.0   0.0  -2.0  -2.0  -2.0  -2.0  -2.0  -2.0  ...   0.0   
3   0.0   2.0   2.0   0.0  -2.0  -2.0  -2.0  -2.0   0.0   0.0  ...  -2.0   
4  -2.0  -2.0   0.0   0.0  -2.0  -2.0  -2.0   0.0  -2.0   0.0  ...   0.0   

   1992  1993  1994  1995  1996  1997  1998  1999  2000  
0   0.0   0.0   0.0  -2.0   0.0   0.0   2.0  -2.0  -1.0  
1  -2.0   0.0  -2.0  -2.0   2.0   2.0   0.0  -2.0   1.0  
2  -2.0   2.0  -2.0   0.0  -2.0  -2.0  -2.0  -2.0  -1.0  
3  -2.0   2.0  -2.0  -2.0   0.0  -2.0   0.0  -2.0   1.0  
4  -2.0  -2.0  -2.0   0.0   0.0  -2.0  -2.0   0.0  -1.0  

[5 rows x 2001 columns]

Number of samples per label type:
2000
-1.0    40
 1.0    22
Name: count, dtype: int

In [28]:
# Required imports
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Split data into features (X) and labels (y)
X = data.iloc[:, :-1]  # All columns except the last column
y = data.iloc[:, -1]   # The last column is assumed to be the target label

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features (Lasso benefits from feature scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Lasso for feature selection (using alpha as regularization strength)
lasso2 = Lasso(alpha=0.01)  # You can adjust alpha for stronger/less regularization
lasso1 = Lasso(alpha = float(input('Enter ALPHA value:')))
# len(selected_features)

lasso1.fit(X_train_scaled, y_train)
lasso2.fit(X_train_scaled, y_train)
# Get the selected features (non-zero coefficients)
selected_features = np.where(lasso1.coef_ != 0)[0]
selected_features2 = np.where(lasso2.coef_ != 0)[0]
print(f"Selected {len(selected_features)} features out of {X_train.shape[1]}")

# Reduce the dataset to selected features
X_train_reduced = X_train_scaled[:, selected_features]
X_test_reduced = X_test_scaled[:, selected_features]

# Apply Decision Tree Classifier on reduced data
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train_reduced, y_train)
y_pred_dt = dt_classifier.predict(X_test_reduced)

# Apply Logistic Regression on reduced data
logreg_classifier = LogisticRegression(random_state=42)
logreg_classifier.fit(X_train_reduced, y_train)
y_pred_logreg = logreg_classifier.predict(X_test_reduced)

# Evaluation for Decision Tree
print("Decision Tree Evaluation:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_dt))
print("Classification Report:")
print(classification_report(y_test, y_pred_dt, target_names=["Class 0", "Class 1"]))

# Evaluation for Logistic Regression
print("\nLogistic Regression Evaluation:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_logreg))
print("Classification Report:")
print(classification_report(y_test, y_pred_logreg, target_names=["Class 0", "Class 1"]))



Selected 67 features out of 2000
Decision Tree Evaluation:
Confusion Matrix:
[[9 1]
 [5 4]]
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.64      0.90      0.75        10
     Class 1       0.80      0.44      0.57         9

    accuracy                           0.68        19
   macro avg       0.72      0.67      0.66        19
weighted avg       0.72      0.68      0.67        19


Logistic Regression Evaluation:
Confusion Matrix:
[[9 1]
 [2 7]]
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.82      0.90      0.86        10
     Class 1       0.88      0.78      0.82         9

    accuracy                           0.84        19
   macro avg       0.85      0.84      0.84        19
weighted avg       0.85      0.84      0.84        19



In [31]:
overlap_features = list(set(selected_features) & set(selected_features2))
# print(f"Alpha value of lasso2: {lasso2.alpha}")
# print(f"Alpha value of lasso1: {lasso1.alpha}")
print(f"Length of Features with ALPHA Value {lasso1.alpha}:",len(selected_features))
print(f"Length of Features with ALPHA Value {lasso2.alpha}:",len(selected_features2))
print(len(overlap_features),': ',overlap_features)

Length of Features with ALPHA Value 0.001: 67
Length of Features with ALPHA Value 0.01: 40
37 :  [2, 515, 779, 140, 1548, 142, 1039, 1676, 159, 160, 1057, 1569, 1059, 1317, 550, 553, 809, 179, 1717, 1334, 56, 697, 187, 1596, 1220, 1608, 723, 342, 599, 988, 869, 1255, 1643, 1901, 110, 123, 764]


In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression



# Split data into features (X) and labels (y)
X = data.iloc[:, :-1]  # All columns except the last column
y = data.iloc[:, -1]   # The last column is assumed to be the target label

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Use Logistic Regression as the estimator for RFE
logreg = LogisticRegression(random_state=42)

# Apply Recursive Feature Elimination (RFE)
rfe = RFE(estimator=logreg, n_features_to_select=40)  # Select top 10 features
rfe.fit(X_train_scaled, y_train)

# Get the selected features
selected_features_rfe = np.where(rfe.support_ == True)[0]

# Output the number of selected features
print(f"Number of selected features: {len(selected_features_rfe)}")
print(f"Selected features indices: {selected_features_rfe}")


Number of selected features: 40
Selected features indices: [   2   84  123  140  244  266  512  515  553  599  641  723  738  764
  779  896  988 1039 1059 1063 1220 1255 1311 1334 1345 1386 1422 1441
 1472 1484 1513 1548 1569 1581 1596 1643 1670 1770 1771 1899]


In [33]:
# Reduce the dataset to selected features
X_train_reduced = X_train_scaled[:, selected_features_rfe]
X_test_reduced = X_test_scaled[:, selected_features_rfe]

# Apply Decision Tree Classifier on reduced data
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train_reduced, y_train)
y_pred_dt = dt_classifier.predict(X_test_reduced)

# Apply Logistic Regression on reduced data
logreg_classifier = LogisticRegression(random_state=42)
logreg_classifier.fit(X_train_reduced, y_train)
y_pred_logreg = logreg_classifier.predict(X_test_reduced)

# Evaluation for Decision Tree
print("Decision Tree Evaluation:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_dt))
print("Classification Report:")
print(classification_report(y_test, y_pred_dt, target_names=["Class 0", "Class 1"]))

# Evaluation for Logistic Regression
print("\nLogistic Regression Evaluation:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_logreg))
print("Classification Report:")
print(classification_report(y_test, y_pred_logreg, target_names=["Class 0", "Class 1"]))

Decision Tree Evaluation:
Confusion Matrix:
[[8 2]
 [3 6]]
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.73      0.80      0.76        10
     Class 1       0.75      0.67      0.71         9

    accuracy                           0.74        19
   macro avg       0.74      0.73      0.73        19
weighted avg       0.74      0.74      0.74        19


Logistic Regression Evaluation:
Confusion Matrix:
[[8 2]
 [3 6]]
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.73      0.80      0.76        10
     Class 1       0.75      0.67      0.71         9

    accuracy                           0.74        19
   macro avg       0.74      0.73      0.73        19
weighted avg       0.74      0.74      0.74        19



In [34]:
# Required imports
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Assuming data is loaded as pandas DataFrame
# Split data into features (X) and labels (y)
X = data.iloc[:, :-1]  # All columns except the last column
y = data.iloc[:, -1]   # The last column is assumed to be the target label

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features (Lasso benefits from feature scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Lasso for feature selection (using alpha as regularization strength)
lasso = Lasso(alpha=0.005)  # You can adjust alpha for stronger/less regularization
lasso.fit(X_train_scaled, y_train)

# Get the selected features (non-zero coefficients)
selected_features = np.where(lasso.coef_ != 0)[0]
print(f"Selected {len(selected_features)} features out of {X_train.shape[1]}")

# Reduce the dataset to selected features
X_train_reduced = X_train_scaled[:, selected_features]
X_test_reduced = X_test_scaled[:, selected_features]

### Apply Decision Tree Classifier on reduced data
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train_reduced, y_train)
y_pred_dt = dt_classifier.predict(X_test_reduced)

### Apply Logistic Regression on reduced data
logreg_classifier = LogisticRegression(random_state=42)
logreg_classifier.fit(X_train_reduced, y_train)
y_pred_logreg = logreg_classifier.predict(X_test_reduced)

### Apply Random Forest Classifier on reduced data
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_reduced, y_train)
y_pred_rf = rf_classifier.predict(X_test_reduced)

### Apply Support Vector Machine (SVM) Classifier on reduced data
svm_classifier = SVC(random_state=42)
svm_classifier.fit(X_train_reduced, y_train)
y_pred_svm = svm_classifier.predict(X_test_reduced)

# Evaluation for Decision Tree
print("Decision Tree Evaluation:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_dt))
print("Classification Report:")
print(classification_report(y_test, y_pred_dt, target_names=["Class 0", "Class 1"]))

# Evaluation for Logistic Regression
print("\nLogistic Regression Evaluation:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_logreg))
print("Classification Report:")
print(classification_report(y_test, y_pred_logreg, target_names=["Class 0", "Class 1"]))

# Evaluation for Random Forest
print("\nRandom Forest Evaluation:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))
print("Classification Report:")
print(classification_report(y_test, y_pred_rf, target_names=["Class 0", "Class 1"]))

# Evaluation for SVM
print("\nSupport Vector Machine (SVM) Evaluation:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_svm))
print("Classification Report:")
print(classification_report(y_test, y_pred_svm, target_names=["Class 0", "Class 1"]))


Selected 47 features out of 2000
Decision Tree Evaluation:
Confusion Matrix:
[[9 1]
 [4 5]]
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.69      0.90      0.78        10
     Class 1       0.83      0.56      0.67         9

    accuracy                           0.74        19
   macro avg       0.76      0.73      0.72        19
weighted avg       0.76      0.74      0.73        19


Logistic Regression Evaluation:
Confusion Matrix:
[[9 1]
 [2 7]]
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.82      0.90      0.86        10
     Class 1       0.88      0.78      0.82         9

    accuracy                           0.84        19
   macro avg       0.85      0.84      0.84        19
weighted avg       0.85      0.84      0.84        19


Random Forest Evaluation:
Confusion Matrix:
[[8 2]
 [2 7]]
Classification Report:
              precision    recall  f1-score   support

    